## ChoiceModels usage demo

Sam Maurer, October 10, 2016

In [1]:
import choicemodels
import numpy as np

### Binary Logit

In [6]:
# Set up estimation data

endog = np.random.randint(2, size=50)  # 50x1 vector of random 0's and 1's
exog = np.random.rand(50, 5)  # 50x5 matrix of random floats

In [7]:
# Estimate a model

m = choicemodels.Logit(endog, exog)
results = m.fit()

Optimization terminated successfully.
         Current function value: 0.653539
         Iterations 5


In [8]:
# Show estimation results

print(results.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   50
Model:                          Logit   Df Residuals:                       45
Method:                           MLE   Df Model:                            4
Date:                Fri, 07 Oct 2016   Pseudo R-squ.:                 0.05277
Time:                        15:33:54   Log-Likelihood:                -32.677
converged:                       True   LL-Null:                       -34.497
                                        LLR p-value:                    0.4568
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             0.1940      1.073      0.181      0.856        -1.908     2.296
x2             1.7003      0.953      1.785      0.074        -0.167     3.567
x3            -0.6705      1.066     -0.629      0.5

### Multinomial Logit